In [1]:
import pandas as pd;

#Load dataset and examine dataset, rename columns to questions and answers

df=pd.read_csv("FAQ_MachineLearningInterview_com.csv");
df.columns=["questions","answers"];

df

,questions,answers
0,What does the job hunting experience look like ?,Job hunting experience involves networking to ...
1,Any insights you can offer about the DS job ma...,"There are many kinds of roles, data scientist,..."
2,What’s the impact of Covid on hiring for DS ro...,Hiring is going to slow down. First in small c...
3,What skills and qualities do employers look fo...,The following are some skills employers usuall...
4,Do employers look for an advanced ML degree?,For more senior roles: People typically look f...
5,How does a typical day of a data scientist loo...,Here are some tasks in the typical day of a da...
6,Is preparation of algorithms and data structur...,Yes. In many data science interviews (ML Scien...
7,What is the mathematical background required t...,The following three are the basic building blo...
8,What are the various rounds in a data scientis...,Usually the data science interview has a subse...
9,What level of proficiency is needed for a data...,"Needs to be reasonably proficient. Again, a da..."


In [2]:
import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
    #sent_stemmed='';
    #for word in sentence.split():
    #    sent_stemmed+=' '+st.stem(word) 
    #sentence=sent_stemmed
    
    return sentence
                    
def get_cleaned_sentences(df,stopwords=False):    
    sents=df[["questions"]];
    cleaned_sentences=[]

    for index,row in df.iterrows():
        #print(index,row)
        cleaned=clean_sentence(row["questions"],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);
    

['job hunting experience look like', 'insights offer ds job market', 'whats impact covid hiring ds roles', 'skills qualities employers look data scientist', 'employers look advanced ml degree', 'typical day data scientist look like', 'preparation algorithms data structures needed data science interview', 'mathematical background required data scientist', 'rounds data scientist interview', 'level proficiency needed data scientist coding']


['what does the job hunting experience look like ', 'any insights you can offer about the ds job market ', 'whats the impact of covid on hiring for ds roles', 'what skills and qualities do employers look for in a data scientist', 'do employers look for an advanced ml degree', 'how does a typical day of a data scientist look like', 'is preparation of algorithms and data structures needed for a data science interview ', 'what is the mathematical background required to be a data scientist ', 'what are the various rounds in a data scientist interview ', 

In [3]:
import numpy

sentences=cleaned_sentences_with_stopwords
#sentences=cleaned_sentences

# Split it by white space 
sentence_words = [[word for word in document.split() ]
         for document in sentences]

from gensim import corpora

dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
    print(key, ' : ', value)

import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)

#question_orig="do I need to learn algorithms to be a data scientist ?";
question_orig="What does a data scientist usually do ?"
question=clean_sentence(question_orig,stopwords=False);
question_embedding = dictionary.doc2bow(question.split())


print("\n\n",question,"\n",question_embedding)

0  :  does
1  :  experience
2  :  hunting
3  :  job
4  :  like
5  :  look
6  :  the
7  :  what
8  :  about
9  :  any
10  :  can
11  :  ds
12  :  insights
13  :  market
14  :  offer
15  :  you
16  :  covid
17  :  for
18  :  hiring
19  :  impact
20  :  of
21  :  on
22  :  roles
23  :  whats
24  :  a
25  :  and
26  :  data
27  :  do
28  :  employers
29  :  in
30  :  qualities
31  :  scientist
32  :  skills
33  :  advanced
34  :  an
35  :  degree
36  :  ml
37  :  day
38  :  how
39  :  typical
40  :  algorithms
41  :  interview
42  :  is
43  :  needed
44  :  preparation
45  :  science
46  :  structures
47  :  background
48  :  be
49  :  mathematical
50  :  required
51  :  to
52  :  are
53  :  rounds
54  :  various
55  :  coding
56  :  level
57  :  proficiency
what does the job hunting experience look like 
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
any insights you can offer about the ds job market 
[(3, 1), (6, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (

In [4]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0];
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim;
            index_sim=index;
       
    print("\n")
    print("Question: ",question)
    print("\n");
    print("Retrieved: ",FAQdf.iloc[index_sim,0]) 
    print(FAQdf.iloc[index_sim,1])        
    
retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences);

0 1.0 what does the job hunting experience look like 
1 0.31622776601683794 any insights you can offer about the ds job market 
2 0.1643989873053573 whats the impact of covid on hiring for ds roles
3 0.19611613513818404 what skills and qualities do employers look for in a data scientist
4 0.19611613513818404 do employers look for an advanced ml degree
5 1.0 how does a typical day of a data scientist look like
6 0.058722021951470346 is preparation of algorithms and data structures needed for a data science interview 
7 0.1643989873053573 what is the mathematical background required to be a data scientist 
8 0.1643989873053573 what are the various rounds in a data scientist interview 
9 0.1414213562373095 what level of proficiency is needed for a data scientist in coding


Question:  what does a data scientist usually do 


Retrieved:  What does the job hunting experience look like ?
Job hunting experience involves networking to get in touch with the right people in various companies, ap

In [5]:
from gensim.models import Word2Vec 
import gensim.downloader as api


glove_model=None;
try:
    glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:            
    glove_model = api.load('glove-twitter-25')
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")
    
v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:            
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved glove model")

w2vec_embedding_size=len(v2w_model['computer']);
glove_embedding_size=len(glove_model['computer']);

Loaded glove model
Loaded w2v model


In [6]:
def getWordVec(word,model):
        samp=model['computer'];
        vec=[0]*len(samp);
        try:
                vec=model[word];
        except:
                vec=[0]*len(samp);
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('computer', embeddingmodel);
        vec=numpy.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+numpy.array(getWordVec(word,embeddingmodel));
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)

In [7]:
#With w2Vec

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

0 0.42883351712089035 job hunting experience look like
1 0.3390023810903811 insights offer ds job market
2 0.2992552732030833 whats impact covid hiring ds roles
3 0.5991923709091536 skills qualities employers look data scientist
4 0.2836109001421265 employers look advanced ml degree
5 0.7728937373489242 typical day data scientist look like
6 0.6020050170744113 preparation algorithms data structures needed data science interview
7 0.6440332904913527 mathematical background required data scientist
8 0.5696568380249727 rounds data scientist interview
9 0.592059380480089 level proficiency needed data scientist coding


Question:  what does a data scientist usually do 


Retrieved:  How does a typical day of a data scientist look like?
Here are some tasks in the typical day of a data scientist:

Make a plan for the day
Look at data, what clean up is required, figure out what models can be built
Talk to various stakeholders about what modeling is possible and help them narrow down to somethi

In [8]:
#With Glove

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,glove_model));
    
question_embedding=getPhraseEmbedding(question,glove_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

0 0.9206842895542597 job hunting experience look like
1 0.774016663828011 insights offer ds job market
2 0.8574905678154825 whats impact covid hiring ds roles
3 0.8762150927509162 skills qualities employers look data scientist
4 0.8210408780157618 employers look advanced ml degree
5 0.9545922769035372 typical day data scientist look like
6 0.8004428870598751 preparation algorithms data structures needed data science interview
7 0.7939550540301473 mathematical background required data scientist
8 0.8481202624655193 rounds data scientist interview
9 0.7833404988245978 level proficiency needed data scientist coding


Question:  what does a data scientist usually do 


Retrieved:  How does a typical day of a data scientist look like?
Here are some tasks in the typical day of a data scientist:

Make a plan for the day
Look at data, what clean up is required, figure out what models can be built
Talk to various stakeholders about what modeling is possible and help them narrow down to somethin

In [14]:
#With BOW, word2Vec and BERT with stopwords removed
question_new="Is it required to have background in  algorithms and complexity for data scientist roles"
#question=clean_sentence("Is algorithms and complexity required for a data science role  ?",stopwords=True);

question=clean_sentence(question_new,stopwords=True);
cleaned_sentences=get_cleaned_sentences(df,stopwords=True)

#Try  BOW
#question_embedding = dictionary.doc2bow(question.lower().split())
#print("\n\n With BOW\n\n")
#retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df);

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

print("\n With Word2Vec\n\n")
retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);


retrieveAndPrintFAQAnswer(question_embedding,sent_bertphrase_embeddings,df, cleaned_sentences);


 With Word2Vec


0 0.3106412106443036 job hunting experience look like
1 0.38500234678021006 insights offer ds job market
2 0.397660092116415 whats impact covid hiring ds roles
3 0.6333366013188841 skills qualities employers look data scientist
4 0.3142924883428163 employers look advanced ml degree
5 0.5751774649060291 typical day data scientist look like
6 0.7404647869018983 preparation algorithms data structures needed data science interview
7 0.8319881968493149 mathematical background required data scientist
8 0.5246081968710783 rounds data scientist interview
9 0.7368409623533582 level proficiency needed data scientist coding


Question:  required background algorithms complexity data scientist roles


Retrieved:  What is the mathematical background required to be a data scientist ?
The following three are the basic building blocks in terms of data science math background: Linear Algebra, Probability and Statistics and Calculus and optimization.


Question:  required background al